## 学校事故データ分析

日本スポーツ振興センターの公開情報よりデータをダウンロード<br>
https://www.jpnsport.go.jp/anzen/anzen_school/anzen_school/tabid/822/Default.aspx<br>
事故の発生状況から「登下校」「授業と休憩時間」「部活動」「学校行事」「その他」の５つに分類する<br>

In [1]:
import pandas as pd

# ファイル読み込み
df = pd.read_excel("sibou(2025.01.31).xlsx")

# 中身の確認（どの列に事故状況が書かれているか）
print(df.columns)
print(df.head(3))

Index(['和暦', '給付年度', '記号', '重複事例', '種別', '被災学校種', '被災学年', '性別', '場合別1', '場合別2',
       '競技種目', '通学方法', '発生場所1', '発生場所2', '遊具等', '災害発生時の状況'],
      dtype='object')
   和暦  給付年度    記号  重複事例        種別 被災学校種  被災学年 性別  場合別1      場合別2  競技種目 通学方法  \
0  平成    17  17死1  17供1        溺死     小     1  男  各教科等  体育（保健体育）    水泳  NaN   
1  平成    17  17死2   NaN  中枢神経系突然死     小     1  女  各教科等  体育（保健体育）  鉄棒運動  NaN   
2  平成    17  17死3   NaN    心臓系突然死     小     6  女  各教科等  体育（保健体育）  短距離走  NaN   

             発生場所1       発生場所2  遊具等  \
0          学校外（園外）          道路  NaN   
1  学校内・校舎外（園内・園舎外）  運動場・校庭（園庭）   鉄棒   
2  学校内・校舎外（園内・園舎外）  運動場・校庭（園庭）  NaN   

                                            災害発生時の状況  
0  体育授業中、近くの温水プールでの水泳学習を終え、準備のできた児童から学校へ戻っていたが、当時...  
1  体育の授業中、低鉄棒の上に腰をかける体勢から飛び降りる運動をしていた際、鉄棒から前向きに落下...  
2  体育の授業中、運動会の１００メートル走の練習として本生徒が１００mを走った際、ゴールした後に...  


In [2]:
print(df["場合別1"].unique())

['各教科等' '特別活動（除学校行事）' '学校行事' '課外指導' '休憩時間' '通学中' '保育中' '寄宿舎にあるとき'
 '技能連携授業中']


In [3]:
print(df["場合別2"].unique())

['体育（保健体育）' '総合的な学習の時間' '給食指導' '文化的行事' '水泳指導' '休憩時間中' '昼食時休憩時間中'
 '始業前の特定時間中' '下校（降園）中' 'その他の教科' '学級（ホームルーム）活動' 'その他' '修学旅行' '体育的部活動'
 '文化的部活動' '授業終了後の特定時間中' '登校（登園）中' '競技大会・球技大会' 'その他集団宿泊的行事' nan '日常の清掃'
 'その他儀式的行事' '運動会・体育祭' 'その他学級活動' '児童（生徒・学生）会活動' '大掃除' '通学（通園）に準ずるとき'
 'その他健康安全・体育的行事' '臨海学校' '理科' '遠足' '図画工作（美術）' '家庭（技術・家庭）' '朝会' '工業' '自立活動']


In [4]:
def map_by_case2(value):
    if pd.isnull(value):
        return "その他"
    
    # 授業・休憩系
    if value in ['体育（保健体育）', '理科', '図画工作（美術）', '家庭（技術・家庭）', 
                 '総合的な学習の時間', 'その他の教科', '学級（ホームルーム）活動',
                 '給食指導', '日常の清掃', '大掃除', '休憩時間中', '昼食時休憩時間中']:
        return "授業と休憩時間"
    
    # 登下校系
    elif value in ['登校（登園）中', '下校（降園）中', '通学（通園）に準ずるとき']:
        return "登下校"
    
    # 学校行事系
    elif value in ['修学旅行', '遠足', '臨海学校', '運動会・体育祭', '文化的行事',
                   'その他健康安全・体育的行事', 'その他儀式的行事', 'その他学級活動', 
                   '児童（生徒・学生）会活動', '朝会', 'その他集団宿泊的行事']:
        return "学校行事"
    
    # 部活動
    elif value in ['体育的部活動', '文化的部活動', '競技大会・球技大会']:
        return "部活動"
    
    # その他（分類不能 or 特定時間など）
    elif value in ['授業終了後の特定時間中', '始業前の特定時間中', 'その他', '自立活動']:
        return "その他"

    else:
        return "その他"

# 新しい分類列を追加
df["分類カテゴリ_場合別2"] = df["場合別2"].apply(map_by_case2)

# 分布を確認
print(df["分類カテゴリ_場合別2"].value_counts(dropna=False))


分類カテゴリ_場合別2
授業と休憩時間    325
部活動        270
登下校        265
その他        223
学校行事        68
Name: count, dtype: int64


In [5]:
df

,和暦,給付年度,記号,重複事例,種別,被災学校種,被災学年,性別,場合別1,場合別2,競技種目,通学方法,発生場所1,発生場所2,遊具等,災害発生時の状況,分類カテゴリ_場合別2
0,平成,17,17死1,17供1,溺死,小,1,男,各教科等,体育（保健体育）,水泳,NaN,学校外（園外）,道路,NaN,体育授業中、近くの温水プールでの水泳学習を終え、準備のできた児童から学校へ戻っていたが、当時...,授業と休憩時間
1,平成,17,17死2,NaN,中枢神経系突然死,小,1,女,各教科等,体育（保健体育）,鉄棒運動,NaN,学校内・校舎外（園内・園舎外）,運動場・校庭（園庭）,鉄棒,体育の授業中、低鉄棒の上に腰をかける体勢から飛び降りる運動をしていた際、鉄棒から前向きに落下...,授業と休憩時間
2,平成,17,17死3,NaN,心臓系突然死,小,6,女,各教科等,体育（保健体育）,短距離走,NaN,学校内・校舎外（園内・園舎外）,運動場・校庭（園庭）,NaN,体育の授業中、運動会の１００メートル走の練習として本生徒が１００mを走った際、ゴールした後に...,授業と休憩時間
3,平成,17,17死4,NaN,心臓系突然死,小,6,男,各教科等,体育（保健体育）,短距離走,NaN,学校内・校舎外（園内・園舎外）,運動場・校庭（園庭）,NaN,体育の時間に、リレー（トラック半周５５ｍ）を自分の走れる速さで走っていたところ、第２コーナー...,授業と休憩時間
4,平成,17,17死5,NaN,心臓系突然死,小,5,女,各教科等,総合的な学習の時間,NaN,NaN,学校内・校舎内（園内・園舎内）,実習実験室,NaN,調理実習中、食器を洗っていた際、突然倒れた。倒れたときは、全身が震えすぐにチアノーゼがあらわ...,授業と休憩時間
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,令和,5,2023死32,NaN,全身打撲,高専,5,男,通学中,下校（降園）中,NaN,鉄道,学校外（園外）,その他,NaN,下校中、激しい腹痛でふらつき、駅のホームから転落したところを電車にひかれ、同日死亡した。,登下校
1147,令和,5,2023死33,NaN,頭部外傷,特高,2,男,特別活動（除学校行事）,学級（ホームルーム）活動,NaN,NaN,学校内・校舎内（園内・園舎内）,その他,NaN,学級（ホームルーム）活動中、トイレから帰ってこない本生徒を教員が探したところ、２階ひさしに出...,授業と休憩時間
1148,令和,5,2023死34,NaN,心臓系突然死,特高,2,男,学校行事,修学旅行,NaN,NaN,学校外（園外）,その他,NaN,修学旅行中、２日目の夕方、体調不良のため部屋で休んでいた際、急にチアノーゼの症状が出て呼吸し...,学校行事
1149,令和,5,2023死35,NaN,窒息死（溺死以外）,保,2,男,保育中,NaN,NaN,NaN,学校内・校舎外（園内・園舎外）,運動場・校庭（園庭）,総合遊具・アスレティック,保育中、園庭の大型遊具で遊んでいた際、遊具内に設置されていた柵と天井の隙間に首が挟まっている...,その他


In [6]:
# ファイルとして保存（例：分類済み_死亡.xlsx）
df.to_excel("分類済み_死亡_場合別2.xlsx", index=False)

### 障害

In [7]:
import pandas as pd

# ファイル読み込み
df = pd.read_excel("shougai(2025.01.31).xlsx")

# 中身の確認（どの列に事故状況が書かれているか）
print(df.columns)
print(df.head(3))

Index(['和暦', '給付年度', '記号', '種別', '被災学校種', '被災学年', '性別', '場合別1', '場合別2', '競技種目',
       '通学方法', '発生場所1', '発生場所2', '遊具等', '災害発生時の状況'],
      dtype='object')
   和暦  給付年度    記号            種別 被災学校種  被災学年 性別  場合別1 場合別2 競技種目 通学方法  \
0  平成    17  17障1  外貌・露出部分の醜状障害     小     4  男  各教科等   理科  NaN  NaN   
1  平成    17  17障2  外貌・露出部分の醜状障害     小     5  女  各教科等   理科  NaN  NaN   
2  平成    17  17障3  外貌・露出部分の醜状障害     小     6  男  各教科等   理科  NaN  NaN   

             発生場所1       発生場所2  遊具等  \
0  学校内・校舎外（園内・園舎外）  運動場・校庭（園庭）  NaN   
1  学校内・校舎内（園内・園舎内）       実習実験室  NaN   
2  学校内・校舎内（園内・園舎内）       実習実験室  NaN   

                                            災害発生時の状況  
0  理科の授業中、運動場で熱気球を飛ばす実験をしていた。黒いビニール袋に針金を通して熱気球を作り...  
1  理科の実験中、他の児童が砂糖を溶かした容器が熱くて持ちきれなくなり、手を離した際、それが右手...  
2  理科実験中に、ペットボトルの中のアルコールに引火し、その勢いで炎と共にアルコールが射出し、左...  


In [8]:
print(df["場合別2"].unique())

['理科' '図画工作（美術）' '家庭（技術・家庭）' '体育（保健体育）' 'その他の教科' 'その他学級活動' '給食指導' '日常の清掃'
 '学級（ホームルーム）活動' '文化的クラブ活動' 'その他健康安全・体育的行事' '修学旅行' 'その他集団宿泊的行事' '大掃除'
 '勤労生産・奉仕的行事' '体育的部活動' '水泳指導' '休憩時間中' '昼食時休憩時間中' '始業前の特定時間中' '授業終了後の特定時間中'
 nan '登校（登園）中' '下校（降園）中' '児童（生徒・学生）会活動' '文化的行事' '運動会・体育祭' '競技大会・球技大会'
 '通学（通園）に準ずるとき' '農業' '自立活動' 'その他' '文化的部活動' '総合的な学習の時間' '体育的クラブ活動' '遠足'
 '工業' 'その他儀式的行事' '林間学校' '道徳']


In [9]:
def classify_case2(value):
    if pd.isnull(value):
        return "その他"
    
    # 授業と休憩時間
    if value in ['理科', '図画工作（美術）', '家庭（技術・家庭）', '体育（保健体育）',
                 'その他の教科', 'その他学級活動', '給食指導', '日常の清掃', '水泳指導',
                 '学級（ホームルーム）活動', '休憩時間中', '昼食時休憩時間中',
                 '始業前の特定時間中', '授業終了後の特定時間中', '児童（生徒・学生）会活動',
                 '農業', '自立活動', '総合的な学習の時間', '工業', 'その他儀式的行事', '道徳']:
        return "授業と休憩時間"
    
    # 部活動
    elif value in ['文化的クラブ活動', '体育的部活動', '文化的部活動', '体育的クラブ活動']:
        return "部活動"
    
    # 学校行事
    elif value in ['その他健康安全・体育的行事', '修学旅行', 'その他集団宿泊的行事', '大掃除',
                   '勤労生産・奉仕的行事', '文化的行事', '運動会・体育祭', '競技大会・球技大会',
                   '遠足', '林間学校']:
        return "学校行事"
    
    # 登下校
    elif value in ['登校（登園）中', '下校（降園）中', '通学（通園）に準ずるとき']:
        return "登下校"
    
    # それ以外
    else:
        return "その他"

In [10]:
df["分類カテゴリ_場合別2"] = df["場合別2"].apply(classify_case2)

In [11]:
# 分類の分布を確認
print(df["分類カテゴリ_場合別2"].value_counts(dropna=False))

分類カテゴリ_場合別2
授業と休憩時間    3440
部活動        2830
登下校         590
その他         465
学校行事        357
Name: count, dtype: int64


In [12]:
# ファイルとして保存（例：分類済み_障害.xlsx）
df.to_excel("分類済み_障害_場合別2.xlsx", index=False)

In [13]:
# 場合別1が「保育中」の行は、分類を「授業と休憩時間」に上書き
df.loc[df["場合別1"] == "保育中", "分類カテゴリ_場合別2"] = "授業と休憩時間"

In [14]:
# 保育中が正しく上書きされたか確認
print(df[df["場合別1"] == "保育中"]["分類カテゴリ_場合別2"].value_counts())

分類カテゴリ_場合別2
授業と休憩時間    380
Name: count, dtype: int64


In [15]:
print(df["分類カテゴリ_場合別2"].value_counts(dropna=False))

分類カテゴリ_場合別2
授業と休憩時間    3820
部活動        2830
登下校         590
学校行事        357
その他          85
Name: count, dtype: int64


In [16]:
# ファイルとして保存（例：分類済み_障害.xlsx）
df.to_excel("改_分類済み_障害_場合別2.xlsx", index=False)

### 障害はこのデータで、死亡は手作業で確認をした上で再びインポート

In [17]:
import pandas as pd

# ファイル読み込み
df_shibou = pd.read_excel("変更後_sibou(2025.01.31).xlsx")

# 中身の確認（どの列に事故状況が書かれているか）
print(df_shibou.columns)
print(df_shibou.head(3))

Index(['和暦', '給付年度', '記号', '重複事例', '種別', '被災学校種', '被災学年', '性別', '場合別1', '場合別2',
       '競技種目', '通学方法', '発生場所1', '発生場所2', '遊具等', '災害発生時の状況', '分類カテゴリ_場合別2'],
      dtype='object')
   和暦  給付年度    記号  重複事例        種別 被災学校種  被災学年 性別  場合別1      場合別2  競技種目 通学方法  \
0  平成    17  17死1  17供1        溺死     小     1  男  各教科等  体育（保健体育）    水泳  NaN   
1  平成    17  17死2   NaN  中枢神経系突然死     小     1  女  各教科等  体育（保健体育）  鉄棒運動  NaN   
2  平成    17  17死3   NaN    心臓系突然死     小     6  女  各教科等  体育（保健体育）  短距離走  NaN   

             発生場所1       発生場所2  遊具等  \
0          学校外（園外）          道路  NaN   
1  学校内・校舎外（園内・園舎外）  運動場・校庭（園庭）   鉄棒   
2  学校内・校舎外（園内・園舎外）  運動場・校庭（園庭）  NaN   

                                            災害発生時の状況 分類カテゴリ_場合別2  
0  体育授業中、近くの温水プールでの水泳学習を終え、準備のできた児童から学校へ戻っていたが、当時...     授業と休憩時間  
1  体育の授業中、低鉄棒の上に腰をかける体勢から飛び降りる運動をしていた際、鉄棒から前向きに落下...     授業と休憩時間  
2  体育の授業中、運動会の１００メートル走の練習として本生徒が１００mを走った際、ゴールした後に...     授業と休憩時間  


In [18]:
print(df_shibou["分類カテゴリ_場合別2"].value_counts(dropna=False))

分類カテゴリ_場合別2
授業と休憩時間    494
登下校        266
部活動        258
学校行事        91
その他         42
Name: count, dtype: int64


In [19]:
# 死亡と障害の分類ごとの件数（Series型として保存）
count_sibou = df_shibou["分類カテゴリ_場合別2"].value_counts()
count_shougai = df["分類カテゴリ_場合別2"].value_counts()

In [20]:
count_sibou

分類カテゴリ_場合別2
授業と休憩時間    494
登下校        266
部活動        258
学校行事        91
その他         42
Name: count, dtype: int64

In [21]:
count_shougai

分類カテゴリ_場合別2
授業と休憩時間    3820
部活動        2830
登下校         590
学校行事        357
その他          85
Name: count, dtype: int64

In [22]:
# 合算（indexを自動で揃えて結合）
total = count_sibou.add(count_shougai, fill_value=0).astype(int)

In [23]:
total

分類カテゴリ_場合別2
その他         127
学校行事        448
授業と休憩時間    4314
登下校         856
部活動        3088
Name: count, dtype: int64

In [24]:
# 合計件数
grand_total = total.sum()

# 割合計算（％表記）
percentage = (total / grand_total * 100).round(1)

In [25]:
# DataFrameにまとめる
summary_df = pd.DataFrame({
    "死亡件数": count_sibou,
    "障害件数": count_shougai,
    "合計": total,
    "割合（%）": percentage
}).fillna(0).astype({"死亡件数": int, "障害件数": int, "合計": int})

In [26]:
import IPython.display as disp
disp.display(summary_df)

,死亡件数,障害件数,合計,割合（%）
分類カテゴリ_場合別2,,,,
その他,42,85,127,1.4
学校行事,91,357,448,5.1
授業と休憩時間,494,3820,4314,48.8
登下校,266,590,856,9.7
部活動,258,2830,3088,35.0


In [27]:
summary_df.to_excel("分類別_死亡と障害_件数割合.xlsx")